In [1]:
import numpy as np
import cv2

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
def extract_video_first_frame(path, use_cv2=True):
    
    cap = cv2.VideoCapture(path)
    
    ret, frame = cap.read()
    
    if use_cv2:
        cv2.imshow('First Frame', frame)

        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
    else:
        new_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        plt.imshow(new_frame)
        plt.axis('off')
        
    return frame

In [3]:
# PATH = './data/videos/rcpc_bimbambum.mp4'
# img = extract_video_first_frame(PATH)

In [4]:
proto_file = './openpose/openpose-master/models/pose/coco/pose_deploy_linevec.prototxt'
weight_file = './openpose/openpose-master/models/pose/coco/pose_iter_440000.caffemodel'

img = cv2.imread('./data/basketball.jpg')
img_copy = img.copy()

In [6]:
frame_width, frame_height = img.shape[1], img.shape[0]
in_height = 368
in_width = 368

pose_net = cv2.dnn.readNetFromCaffe(proto_file, weight_file)
pose_blob = cv2.dnn.blobFromImage(img_copy, scalefactor=1.0/255, size=(in_width, in_height), mean=(0,0,0), swapRB=False, crop=False)
pose_net.setInput(pose_blob)
output = pose_net.forward()

In [7]:
len(output)

1

# Single Point

In [ ]:
i = 0

prob_map = output[0,i,:,:]
prob_map = cv2.resize(prob_map, (frame_width, frame_height))

plt.imshow(cv2.cvtColor(img_copy, cv2.COLOR_BGR2RGB))    
plt.imshow(prob_map, alpha=0.5)
plt.show()

In [ ]:
threshold = 0.1

map_gauss = cv2.GaussianBlur(prob_map, (3,3), 0, 0)
map_mask = np.uint8(map_gauss > threshold)

In [ ]:
contours, _ = cv2.findContours(map_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
body_points = []

for c in contours:
    blob_mask = np.zeros(map_mask.shape)
    blob_mask = cv2.fillConvexPoly(blob_mask, c, 1)
    masked_prob_map = map_gauss * blob_mask
    _, max_val, _, max_loc = cv2.minMaxLoc(masked_prob_map)
    
    body_points.append(max_loc + (prob_map[max_loc[1], max_loc[0]],))

In [ ]:
for i in range(len(body_points)):
    cv2.circle(img_copy, (body_points[i][0], body_points[i][1]), 5, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)

cv2.imshow('Body Points', img_copy)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Multiple Points

In [6]:
img_copy = img.copy()
n_points = 18
key_points_mapping = ['Nose', 'Neck', 'R-Shoulder', 'R-Elbow', 'R-Wrist', 'L-Shoulder', 'L-Elbow', 'L-Wrist', 'R-Hip', 'R-Knee', 'R-Ankle',
                     'L-Hip', 'L-Knee', 'L-Ankle', 'R-Eye', 'L-Eye', 'R-Ear', 'L-Ear']


POSE_PAIRS = [[1,2], [1,5], [2,3], [3,4], [5,6], [6,7],
              [1,8], [8,9], [9,10], [1,11], [11,12], [12,13],
              [1,0], [0,14], [14,16], [0,15], [15,17],
              [2,17], [5,16]]

# index of pafs correspoding to the POSE_PAIRS
# e.g for POSE_PAIR(1,2), the PAFs are located at indices (31,32) of output, Similarly, (1,5) -> (39,40) and so on.
mapIdx = [[31,32], [39,40], [33,34], [35,36], [41,42], [43,44],
          [19,20], [21,22], [23,24], [25,26], [27,28], [29,30],
          [47,48], [49,50], [53,54], [51,52], [55,56],
          [37,38], [45,46]]
 
colors = [[0,100,255], [0,100,255], [0,255,255], [0,100,255], [0,255,255], [0,100,255],
         [0,255,0], [255,200,100], [255,0,255], [0,255,0], [255,200,100], [255,0,255],
         [0,0,255], [255,0,0], [200,200,0], [255,0,0], [200,200,0], [0,0,0]]

In [11]:
def get_key_points(prob_map, threshold=0.1):
    body_points = []
    
    map_gauss = cv2.GaussianBlur(prob_map, (3,3), 0, 0)
    map_mask = np.uint8(map_gauss > threshold)
    
    contours, _ = cv2.findContours(map_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for c in contours:
        blob_mask = np.zeros(map_mask.shape)
        blob_mask = cv2.fillConvexPoly(blob_mask, c, 1)
        masked_prob_map = map_gauss * blob_mask
        _, max_val, _, max_loc = cv2.minMaxLoc(masked_prob_map)

        body_points.append(max_loc + (prob_map[max_loc[1], max_loc[0]],))

    return body_points

In [14]:
frame_width = img_copy.shape[1]
frame_height = img_copy.shape[0]
in_height = 368
in_width = int((in_height/frame_height) * frame_width)

pose_model = cv2.dnn.readNetFromCaffe(proto_file, weight_file)
input_blob = cv2.dnn.blobFromImage(img_copy, 1.0/255, (in_width, in_height), (0,0,0), swapRB=False, crop=False)

pose_model.setInput(input_blob)
output = pose_model.forward()

detected_key_points = []
key_points_list = np.zeros((0,3))
key_point_id = 0

for i in range(n_points):
    prob_map = output[0,i,:,:]
    prob_map = cv2.resize(prob_map, (img_copy.shape[1], img_copy.shape[0]))
    body_points = get_key_points(prob_map)
    print('Key points - {} : {}'.format(key_points_mapping[i], body_points))
    
    key_points_with_id = []
    for j in range(len(body_points)):
        key_points_with_id.append(body_points[j] + (key_point_id,))
        key_points_list = np.vstack([key_points_list, body_points[j]])
        
        key_point_id += 1
        
    detected_key_points.append(key_points_with_id)
    
for i in range(n_points):
    for j in range(len(detected_key_points[i])):
        cv2.circle(img_copy, detected_key_points[i][j][0:2], 5, colors[i], -1, cv2.LINE_AA)
        
cv2.imshow('Key Points', img_copy)
cv2.waitKey(0)
cv2.destroyAllWindows()

Key points - Nose : [(145, 170, 0.24120998), (136, 127, 0.4184789), (408, 128, 0.8520362), (329, 75, 0.11857333), (444, 55, 0.26735687), (206, 48, 0.3328478), (171, 48, 0.5133302)]
Key points - Neck : [(144, 172, 0.2667331), (391, 161, 0.70504475), (135, 137, 0.4335356), (284, 101, 0.36854365), (451, 65, 0.5178138), (206, 57, 0.66114914), (171, 57, 0.71380293), (681, 31, 0.16509917)]
Key points - R-Shoulder : [(162, 179, 0.18160741), (356, 163, 0.68080056), (127, 143, 0.25736088), (286, 74, 0.14741245), (443, 65, 0.48046187), (162, 57, 0.62933254), (205, 57, 0.5204189), (680, 31, 0.109229974)]
Key points - R-Elbow : [(320, 215, 0.73568434), (162, 189, 0.15284778), (434, 75, 0.37344542), (197, 73, 0.34908393), (154, 74, 0.47662666), (321, 56, 0.40663266), (74, 13, 0.5823758)]
Key points - R-Wrist : [(364, 242, 0.86022645), (162, 189, 0.14159219), (100, 118, 0.44911006), (442, 91, 0.29245332), (154, 83, 0.25816432), (206, 74, 0.21006168), (381, 31, 0.683152)]
Key points - L-Shoulder : [(

In [36]:
proto_file = './openpose/openpose-master/models/pose/mpi/pose_deploy_linevec_faster_4_stages.prototxt'
weight_file = './openpose/openpose-master/models/pose/mpi/pose_iter_160000.caffemodel'

frame_width, frame_height = img.shape[1], img.shape[0]
in_height = 368
in_width = 368

pose_net = cv2.dnn.readNetFromCaffe(proto_file, weight_file)
pose_blob = cv2.dnn.blobFromImage(img_copy, scalefactor=1.0/255, size=(in_width, in_height), mean=(0,0,0), swapRB=False, crop=False)
pose_net.setInput(pose_blob)
output = pose_net.forward()

img = cv2.imread('./data/basketball.jpg')
img_copy = img.copy()

h, w = output.shape[2], output.shape[3]
threshold = 0.1
n_points = 44 # MPI Pose

body_points = []
for i in range(n_points):
    probs = output[0,i,:,:]
    min_val, prob, min_loc, point = cv2.minMaxLoc(probs)
    
    if prob > threshold:
        
        x = (frame_width * point[0]) / w
        y = (frame_height * point[1]) / h
        
        cv2.circle(img_copy, (int(x), int(y)), 5, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
        cv2.putText(img_copy, "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 3, lineType=cv2.LINE_AA)
        
        body_points.append((int(x), int(y)))
        
    else:
        body_points.append(None)
        
cv2.imshow('Body Points', img_copy)    
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
body_points